In [1]:
import parse_tfl_json
import google

In [26]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime_cw where tfl_message = 'ok'
"""
df = pd.read_sql(sql, conn)

In [29]:
import json 

for r in df.iterrows():
    index = r[0]
    row = r[1]

    j1 = row["tfl_response"]
    j2 = json.loads(j1)
    
    try:
        journeys = j2["journeys"]
        # Keep only journeys that end in central London
        
        #journey = sorted(journeys, key = lambda x:  x["duration"])[0]

        legs = journey["legs"]

        legs = parse_tfl_json.remove_non_national_rail(legs)

        summary = parse_tfl_json.summarise_legs(legs)
        df.loc[index, "natrail_journey_summary"] = summary

        df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
        df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

        travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
        df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

        lat_lng = parse_tfl_json.lat_lng_from_legs(legs)
        df.loc[index, "depart_lat"] = lat_lng["depart"]["lat"]
        df.loc[index, "depart_lng"] = lat_lng["depart"]["lng"]
        df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
        df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]
    except Exception as e:
        df.loc[index, "error"] = "Parse problem"
    
    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict", "journey_time", "london_mainline_station_ics", "london_mainline_station_text"]]
df2 = df[cols]

In [51]:
df3 = df2.sort_values(["station_name", "natrail_journey_minutes"]).drop_duplicates("station_name")

In [61]:
df_arrivals = df2.drop_duplicates("final_arrival")[["final_arrival", "arrive_lat", "arrive_lng"]]


In [64]:
# Next - need a bikedistance function that takes a lat lng and looks up 

for r in df_arrivals.iterrows():
    index = r[0]
    row = r[1]
    
    lat = row["arrive_lat"]
    lng = row["arrive_lng"]
    
    try:
        cycle_info = get_cycle_info(origin_lat = lat, origin_lng = lng, dest_pc= "E145HP")
    
        df_arrivals.loc[index, "cycle_miles_london"] = cycle_info["miles"]
        df_arrivals.loc[index, "cycle_minutes_london"] = cycle_info["minutes"]
    except Exception as e:
        pass


{'destination_addresses': ['London E14 5HP, UK'], 'origin_addresses': ['22 Great Eastern Rd, London E15 1BB, UK'], 'rows': [{'elements': [{'distance': {'text': '3.9 mi', 'value': 6271}, 'duration': {'text': '16 mins', 'value': 976}, 'status': 'OK'}]}], 'status': 'OK'}
{'destination_addresses': ['London E14 5HP, UK'], 'origin_addresses': ['1 Leake St, Lambeth, London SE1 7NN, UK'], 'rows': [{'elements': [{'distance': {'text': '6.2 mi', 'value': 9925}, 'duration': {'text': '34 mins', 'value': 2060}, 'status': 'OK'}]}], 'status': 'OK'}
{'destination_addresses': ['London E14 5HP, UK'], 'origin_addresses': ['20 York Way, Kings Cross, London N1 9AA, UK'], 'rows': [{'elements': [{'distance': {'text': '5.8 mi', 'value': 9352}, 'duration': {'text': '37 mins', 'value': 2229}, 'status': 'OK'}]}], 'status': 'OK'}
{'destination_addresses': ['London E14 5HP, UK'], 'origin_addresses': ['11 London St, London W2, UK'], 'rows': [{'elements': [{'distance': {'text': '8.3 mi', 'value': 13365}, 'duration': 

In [97]:
df_arrivals = df_arrivals[["final_arrival", "cycle_miles_london", "cycle_minutes_london"]]
df4 = df3.merge(df_arrivals, how = "left", left_on = "final_arrival", right_on = "final_arrival")

In [101]:
keep = ["station_name", "natrail_journey_summary", "depart_lat", "depart_lng", "natrail_journey_minutes","cycle_minutes_london"]
df4 = df4[keep]
df4["total_minutes"] = df4["cycle_minutes_london"] + df4["natrail_journey_minutes"]


In [107]:
# Want to update all the names to be cw specific 
df5 = df4.rename(columns={"depart_lat" : "lat", "depart_lng": "lng", "station_name": "name"})

In [109]:
df5.to_csv("cw_to_map.csv", encoding="utf-8", index=False)

In [110]:
df5.columns

Index(['name', 'natrail_journey_summary', 'lat', 'lng',
       'natrail_journey_minutes', 'cycle_minutes_london', 'total_minutes'],
      dtype='object')

In [120]:
# To do - pick best journey time by total, not just the time on the train.  
# We can filter out suggested journeys where the lat lng is outside of central London

In [124]:
journeys[1]

{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Journey, Tfl.Api.Presentation.Entities',
 'arrivalDateTime': '2017-09-08T10:27:00',
 'duration': 94,
 'legs': [{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Leg, Tfl.Api.Presentation.Entities',
   'arrivalPoint': {'$type': 'Tfl.Api.Presentation.Entities.StopPoint, Tfl.Api.Presentation.Entities',
    'additionalProperties': [],
    'commonName': 'Reading Rail Station',
    'icsCode': '1001554',
    'lat': 0.0,
    'lon': 0.0,
    'naptanId': '910GRDNG4AB',
    'placeType': 'StopPoint',
    'platformName': ''},
   'arrivalTime': '2017-09-08T09:10:00',
   'departurePoint': {'$type': 'Tfl.Api.Presentation.Entities.StopPoint, Tfl.Api.Presentation.Entities',
    'additionalProperties': [],
    'commonName': 'Aldermaston Rail Station',
    'icsCode': '90000110',
    'lat': 51.40196000195,
    'lon': -1.13742385129,
    'naptanId': '910GALDMSTN',
    'placeType': 'StopPoint',
    'platformName': ''},
   'departureTime': '2017-09